In [1]:
import ee
import geemap

In [2]:
PROJECT_ID = 'aifinal-480001' 

try:
    ee.Initialize(project=PROJECT_ID) 
    print("¡Earth Engine inicializado con éxito!")

except Exception as e:
    print(f"Error: {e}")

¡Earth Engine inicializado con éxito!


In [3]:
aoi_coords = [
    [
        [-66.22589320019163, -17.327984858056993],
        [-66.2177308689185, -17.31408528970765],
        [-66.15662368614169, -17.328511336185215],
        [-66.10180370448111, -17.36862452688139],
        [-66.12584949120544, -17.365992696779358],
        [-66.22589320019163, -17.327984858056993]
    ]
]
roi = ee.Geometry.Polygon(aoi_coords)

In [4]:
def obtener_imagen_sentinel(anio, roi):
    collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(roi) \
        .filterDate(f'{anio}-05-01', f'{anio}-09-30') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') 

    image = collection.first()
    
    return image.clip(roi)

In [5]:
imagenes = {
    '2016 (Base)': obtener_imagen_sentinel(2016, roi),
    '2020 (Medio)': obtener_imagen_sentinel(2020, roi),
    '2024 (Reciente)': obtener_imagen_sentinel(2024, roi) 
}

In [6]:
Map = geemap.Map()
Map.centerObject(roi, 13)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] # Rojo, Verde, Azul
}

for nombre, img in imagenes.items():
    try:
        info = img.getInfo() 
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        print(f"Imagen encontrada para {nombre}: Fecha {date}")
        Map.addLayer(img, vis_params, nombre)
    except:
        print(f"No se encontró imagen limpia para {nombre} con los filtros actuales.")

Map.addLayer(roi, {'color': 'red'}, 'AOI Límite')
Map

Imagen encontrada para 2016 (Base): Fecha 2016-07-13
Imagen encontrada para 2020 (Medio): Fecha 2020-05-08
Imagen encontrada para 2024 (Reciente): Fecha 2024-06-06


Map(center=[-17.339545973154127, -66.16847578348244], controls=(WidgetControl(options=['position', 'transparen…